In [6]:
from enum import Enum

In [150]:
class Direction:
    LEFT = '<'
    RIGHT = '>'
    UP = '^'
    DOWN = 'v'
    
class TurnDirection:
    LEFT = 0
    RIGHT = 1
    STRAIGHT = 2

class Vehicle:
    RULES = {
        '/': {
            Direction.LEFT:  Direction.DOWN,
            Direction.RIGHT: Direction.UP,
            Direction.UP:    Direction.RIGHT,
            Direction.DOWN:  Direction.LEFT,
        },
        '\\': {
            Direction.LEFT:  Direction.UP,
            Direction.RIGHT: Direction.DOWN,
            Direction.UP:    Direction.LEFT,
            Direction.DOWN:  Direction.RIGHT,
        },
    }
    
    TURN_RULES = {
        TurnDirection.LEFT: {
            Direction.LEFT:  Direction.DOWN,
            Direction.DOWN:  Direction.RIGHT,
            Direction.RIGHT:  Direction.UP,
            Direction.UP:    Direction.LEFT,
        },
        TurnDirection.RIGHT: {
            Direction.LEFT:  Direction.UP,
            Direction.UP:    Direction.RIGHT,
            Direction.RIGHT:  Direction.DOWN,
            Direction.DOWN:  Direction.LEFT,
        },
    }
    
    CHANGE_TURN_RULES = {
        TurnDirection.LEFT: TurnDirection.STRAIGHT,
        TurnDirection.STRAIGHT: TurnDirection.RIGHT,
        TurnDirection.RIGHT: TurnDirection.LEFT,
    }
    
    def __init__(self, x, y, direction):
        self.x = x
        self.y = y
        self.direction = direction
        self.current_turn = TurnDirection.LEFT
    
    def update(self, world):
        self.update_position()
        cell = world[self.y][self.x]
        if cell == ' ':
            raise ValueError('Vehicle went off the road')
        # intersection
        if cell == '+':
            self.update_intersection()
        elif cell in Vehicle.RULES:
            self.direction = Vehicle.RULES[cell][self.direction]

    def update_intersection(self):
        if self.current_turn != TurnDirection.STRAIGHT:
            self.direction = Vehicle.TURN_RULES[self.current_turn][self.direction]
        self.current_turn = Vehicle.CHANGE_TURN_RULES[self.current_turn]
        
    def update_position(self):
        if self.direction == Direction.LEFT:
            self.x -= 1
        elif self.direction == Direction.RIGHT:
            self.x += 1
        elif self.direction == Direction.UP:
            self.y -= 1
        elif self.direction == Direction.DOWN:
            self.y += 1
        
    def __str__(self):
        return '[{},{}]: {}'.format(self.x, self.y, self.direction)
    
    def __repr__(self):
        return self.__str__()

In [224]:
class World:
    def __init__(self):
        self.world = []
        self.height = 0
        self.width = 0
        self.vehicles = []
        
    def load_from_file(self, file):
        for y, line in enumerate(file.readlines()):
            line = line.replace('\n', '')
            for x, char in enumerate(line):
                if char in set('<>^v'):
                    self.add_vehicle(Vehicle(x, y, char))
            world_line = line.replace('>', '-').replace('<', '-')\
                             .replace('^', '|').replace('v', '|')
            self.world.append(list(world_line))
        self.height = len(self.world)
        self.width = len(self.world[0])
        
    def add_vehicle(self, vehicle):
        self.vehicles.append(vehicle)
    
    # when preventing collisions, vehicle pais
    def update(self, prevent_crashes=False):
        crashed_vehicles = []
        # check for head on collisions
        self.vehicles.sort(key=self.get_global_index)
        for vehicle in self.vehicles:
            if vehicle in crashed_vehicles: # ignore crashed vehicles
                continue
            vehicle.update(self.world)
            other_vehicle = self.check_collisions(vehicle)
            if other_vehicle:
                crashed_vehicles.append(vehicle)
                crashed_vehicles.append(other_vehicle)
            # if there was a crash without collision prevention, exit simulation
            if not prevent_crashes and other_vehicle:
                break
        # if preventing crashes, remove vehicles
        if prevent_crashes:
            for vehicle in crashed_vehicles:
                self.vehicles.remove(vehicle)
        return crashed_vehicles
    
    def get_global_index(self, vehicle):
        index = vehicle.x + vehicle.y*self.width
        return index

    def check_collisions(self, suspect):
        for vehicle in self.vehicles:
            if vehicle is suspect:
                continue
            if vehicle.x == suspect.x and vehicle.y == suspect.y:
                x, y = vehicle.x, vehicle.y
                return vehicle
        return None
    
    def get_current_vehicle_positions(self):
        cache = {}
        for vehicle in self.vehicles:
            if vehicle.x not in cache:
                cache[vehicle.x] = {vehicle.y:vehicle.direction}
            else:
                if vehicle.y not in cache[vehicle.x]:
                    cache[vehicle.x][vehicle.y] = vehicle.direction
                else: # collision occurred
                    cache[vehicle.x][vehicle.y] = 'X'
        return cache
    
    def __str__(self):
        buffer = []
        positions = self.get_current_vehicle_positions()
        for y in range(self.height):
            line = ''
            for x in range(self.width):
                if x in positions and y in positions[x]:
                    line += positions[x][y]
                else:
                    line += self.world[y][x]
            buffer.append(line)
        return '\n'.join(buffer)

In [225]:
def retrieve_world(filepath):
    world = World()
    with open(filepath, 'r') as file:
        world.load_from_file(file)
    return world

In [226]:
filepath = 'data/day13.txt'
#filepath = 'data/day13_mini.txt'

In [227]:
world = retrieve_world(filepath)
collisions = []
current_iter = 0

while len(collisions) <= 0:
    collisions = world.update()
    current_iter += 1
print(collisions[0], current_iter)

[65,73]: > 150


In [228]:
world = retrieve_world(filepath)
current_iter = 0

while len(world.vehicles) > 1:
    collisions = world.update(prevent_crashes=True)
    current_iter += 1
print(world.vehicles[0], current_iter)

[54,66]: < 17145
